## 4. Ensemble

Kaggle score: 

## Run names

In [1]:
import time

project_name = 'TalkingdataAFD2018'
step_name = 'Ensemble'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = '%s_%s_%s' % (project_name, step_name, time_str)
print('run_name: %s' % run_name)
t0 = time.time()

run_name: TalkingdataAFD2018_Ensemble_20180507_223338


## Important params

In [2]:
import random
statistics_type = 'val_roc_mean'

random_num = random.randint(1, 10000)
print('random_num: ', random_num)

random_num:  5998


## Import PKGs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import display

import os
import gc
import zipfile
import h5py
import pickle
import math
from PIL import Image
import shutil

from tqdm import tqdm
from multiprocessing import cpu_count
cpu_amount = cpu_count()

from sklearn.metrics import confusion_matrix, accuracy_score

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Project folders

In [4]:
cwd = os.getcwd()

feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
log_folder = os.path.join(cwd, 'log')
model_folder = os.path.join(cwd, 'model')
output_folder = os.path.join(cwd, 'output')

print('feature_folder: \t\t%s' % feature_folder)
print('input_folder: \t\t\t%s' % input_folder)
print('log_folder: \t\t\t%s' % log_folder)
print('model_folder: \t\t\t%s' % model_folder)
print('output_folder: \t\t\t%s' % output_folder)


train_csv_file = os.path.join(input_folder, 'train.csv')
train_sample_csv_file = os.path.join(input_folder, 'train_sample.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_csv_file = os.path.join(input_folder, 'sample_submission.csv')

print('\ntrain_csv_file: \t\t%s' % train_csv_file)
print('train_sample_csv_file: \t\t%s' % train_sample_csv_file)
print('test_csv_file: \t\t\t%s' % test_csv_file)
print('sample_submission_csv_file: \t%s' % sample_submission_csv_file)

feature_folder: 		/data1/kaggle/talkingdata-adtracking-fraud-detection/feature
input_folder: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/input
log_folder: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/log
model_folder: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/model
output_folder: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/output

train_csv_file: 		/data1/kaggle/talkingdata-adtracking-fraud-detection/input/train.csv
train_sample_csv_file: 		/data1/kaggle/talkingdata-adtracking-fraud-detection/input/train_sample.csv
test_csv_file: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/input/test.csv
sample_submission_csv_file: 	/data1/kaggle/talkingdata-adtracking-fraud-detection/input/sample_submission.csv


## Load predict probability files

In [5]:
ori_proba_files = [
#     { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBClassifier_GSCV_20180429_063128_9748.p', 'kaggle_score': 0.17812},
#     { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBClassifier_GSCV_20180429_101701_date71_9527.p'},
#     { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBClassifier_GSCV_20180429_111325_date71_9584.p'},
#     { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBClassifier_GSCV_20180429_114421_date71_9493.p'},
#     { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBClassifier_GSCV_20180430_074910_date71_9560.p'},
    
#     { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBClassifier_GSCV_20180430_105818_date71_9633.p'},
#     { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBClassifier_GSCV_20180430_092846_date71_9621.p'},
#     { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBClassifier_GSCV_20180430_084725_date71_9604.p'},
#     { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBClassifier_GSCV_20180430_114955_date71_9643.p'},
#     { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBoost_20180506_204229_date100_9722.p'},
    { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBoost_20180506_233145_date100_9772.p'},
    { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBoost_20180507_065745_date100_9768.p'},
    
    { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBoost_20180507_090216_date100_9774.p'},
    { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBoost_20180506_204229_date100_9722.p'},
    { 'file_name': 'proba_TalkingdataAFD2018_FE_XGBClassifier_GSCV_20180501_110046_date7_9680.p'},
    
]

for file in ori_proba_files:
    if os.path.exists(os.path.join(model_folder, file['file_name'])):
        print('File exists: %s' % file['file_name'])
    else:
        print('***File do not exists: %s' % file['file_name'])

File exists: proba_TalkingdataAFD2018_FE_XGBoost_20180506_233145_date100_9772.p
File exists: proba_TalkingdataAFD2018_FE_XGBoost_20180507_065745_date100_9768.p
File exists: proba_TalkingdataAFD2018_FE_XGBoost_20180507_090216_date100_9774.p
File exists: proba_TalkingdataAFD2018_FE_XGBoost_20180506_204229_date100_9722.p
File exists: proba_TalkingdataAFD2018_FE_XGBClassifier_GSCV_20180501_110046_date7_9680.p


In [6]:
%%time
from sklearn.metrics import roc_auc_score

def save_proba(y_val_proba, y_val, y_test_proba, click_ids, file_name):
    print(click_ids[:5])
    if os.path.exists(file_name):
        os.remove(file_name)
        print('File removed: \t%s' % file_name)
    with h5py.File(file_name) as h:
#         h.create_dataset('y_train_proba', data=y_train_proba)
#         h.create_dataset('y_train', data=y_train)
        h.create_dataset('y_val_proba', data=y_val_proba)
        h.create_dataset('y_val', data=y_val)
        h.create_dataset('y_test_proba', data=y_test_proba)
        h.create_dataset('click_ids', data=click_ids)
    print('File saved: \t%s' % file_name)

def load_proba(file_name):
    with h5py.File(file_name, 'r') as h:
#         y_train_proba = np.array(h['y_train_proba'])
#         y_train = np.array(h['y_train'])
        y_val_proba = np.array(h['y_val_proba'])
        y_val = np.array(h['y_val'])
        y_test_proba = np.array(h['y_test_proba'])
        click_ids = np.array(h['click_ids'])
    print('File loaded: \t%s' % file_name)
    print(click_ids[:5])
    
    return y_val_proba, y_val, y_test_proba, click_ids


def get_acc(y_pred, y_proba):
    y_hat = (y_proba>=0.5).astype(int)
    return accuracy_score(y_pred, y_hat)

def get_roc(y_pred, y_proba):
    return roc_auc_score(y_pred, y_proba)


y_train_probas = []
y_trains = []
y_val_probas = []
y_vas = []
y_test_probas = []
for file in ori_proba_files:
    y_proba_file = os.path.join(model_folder, file['file_name'])
    y_val_proba, y_val, y_test_proba, click_ids = load_proba(y_proba_file)
#     print(y_train_proba.shape)
#     print(y_train.shape)
#     y_train_probas.append(y_train_proba)
    print(len(y_val_proba.shape))
    if len(y_val_proba.shape) == 2:
        y_val_proba = y_val_proba[:, 1]
        y_test_proba = y_test_proba[:, 1]
    print(y_val_proba.shape)
    print(y_val.shape)
    print(y_test_proba.shape)
    print(len(click_ids))
    y_val_probas.append(y_val_proba)
    y_test_probas.append(y_test_proba)
#     file['train_acc'] = get_acc(y_train, y_train_proba)
    file['acc_val'] = get_acc(y_val, y_val_proba)
    
#     file['train_roc'] = get_roc(y_train, y_train_proba)
    file['roc_val'] = get_roc(y_val, y_val_proba)

for f in ori_proba_files:
    print(f['file_name'], '%.4f' % f['acc_val'], '%.4f' % f['roc_val'])

File loaded: 	/data1/kaggle/talkingdata-adtracking-fraud-detection/model/proba_TalkingdataAFD2018_FE_XGBoost_20180506_233145_date100_9772.p
[0 1 2 3 4]
1
(9308568,)
(9308568,)
(18790469,)
18790469
File loaded: 	/data1/kaggle/talkingdata-adtracking-fraud-detection/model/proba_TalkingdataAFD2018_FE_XGBoost_20180507_065745_date100_9768.p
[0 1 2 3 4]
1
(9308568,)
(9308568,)
(18790469,)
18790469
File loaded: 	/data1/kaggle/talkingdata-adtracking-fraud-detection/model/proba_TalkingdataAFD2018_FE_XGBoost_20180507_090216_date100_9774.p
[0 1 2 3 4]
1
(9308568,)
(9308568,)
(18790469,)
18790469
File loaded: 	/data1/kaggle/talkingdata-adtracking-fraud-detection/model/proba_TalkingdataAFD2018_FE_XGBoost_20180506_204229_date100_9722.p
[0 1 2 3 4]
1
(9308568,)
(9308568,)
(18790469,)
18790469
File loaded: 	/data1/kaggle/talkingdata-adtracking-fraud-detection/model/proba_TalkingdataAFD2018_FE_XGBClassifier_GSCV_20180501_110046_date7_9680.p
[0 1 2 3 4]
2
(9308568,)
(9308568,)
(18790469,)
18790469
proba_

## Get basic statistical data

In [7]:
ensemble_res = {}

In [8]:
%%time
def get_mean(probas):
    probas_newaxises = []
    for p in probas:
        print('*', end='')
        probas_newaxises.append(p[:, np.newaxis])
    probas_newaxis = np.concatenate(probas_newaxises, axis=-1)
    probas_mean = np.mean(probas_newaxis, axis=-1)
    print('probas_mean.shape: \t', probas_mean.shape)
    return probas_mean

# ensemble_res['train_roc_mean'] = get_roc(y_train ,get_mean(y_train_probas))
ensemble_res['val_roc_mean'] = get_roc(y_val ,get_mean(y_val_probas))
y_test_proba_mean = get_mean(y_test_probas)

# print(ensemble_res)

*****probas_mean.shape: 	 (9308568,)
*****probas_mean.shape: 	 (18790469,)
CPU times: user 3.64 s, sys: 356 ms, total: 4 s
Wall time: 4 s


In [9]:
%%time
def get_min(probas):
    probas_newaxises = []
    for p in probas:
        print('*', end='')
        probas_newaxises.append(p[:, np.newaxis])
    probas_newaxis = np.concatenate(probas_newaxises, axis=-1)
    probas_min = np.min(probas_newaxis, axis=-1)
    print('probas_min.shape: \t', probas_min.shape)
    return probas_min

# ensemble_res['train_roc_min'] = get_roc(y_train ,get_min(y_train_probas))
ensemble_res['val_roc_min'] = get_roc(y_val ,get_min(y_val_probas))
y_test_proba_min = get_min(y_test_probas)

# print(ensemble_res)

*****probas_min.shape: 	 (9308568,)
*****probas_min.shape: 	 (18790469,)
CPU times: user 3.98 s, sys: 356 ms, total: 4.34 s
Wall time: 4.33 s


In [10]:
%%time
def get_max(probas):
    probas_newaxises = []
    for p in probas:
        print('*', end='')
        probas_newaxises.append(p[:, np.newaxis])
    probas_newaxis = np.concatenate(probas_newaxises, axis=-1)
    probas_max = np.max(probas_newaxis, axis=-1)
    print('probas_max.shape: \t', probas_max.shape)
    return probas_max

# ensemble_res['train_roc_max'] = get_roc(y_train ,get_max(y_train_probas))
ensemble_res['val_roc_max'] = get_roc(y_val ,get_max(y_val_probas))
y_test_proba_max = get_max(y_test_probas)

# print(ensemble_res)

*****probas_max.shape: 	 (9308568,)
*****probas_max.shape: 	 (18790469,)
CPU times: user 4.08 s, sys: 312 ms, total: 4.39 s
Wall time: 4.39 s


In [11]:
%%time
def get_median(probas):
    probas_newaxises = []
    for p in probas:
        print('*', end='')
        probas_newaxises.append(p[:, np.newaxis])
    probas_newaxis = np.concatenate(probas_newaxises, axis=-1)
    probas_median = np.median(probas_newaxis, axis=-1)
    print('probas_median.shape: \t', probas_median.shape)
    return probas_median

# ensemble_res['train_roc_median'] = get_roc(y_train ,get_median(y_train_probas))
ensemble_res['val_roc_median'] = get_roc(y_val ,get_median(y_val_probas))
y_test_proba_median = get_median(y_test_probas)

# print(ensemble_res)

*****probas_median.shape: 	 (9308568,)
*****probas_median.shape: 	 (18790469,)
CPU times: user 5.58 s, sys: 556 ms, total: 6.14 s
Wall time: 6.14 s


In [12]:
for i, key in enumerate(ensemble_res.keys()):
#     if i % 2 == 0:
#         print('%-15s \t%.4f' % (key, ensemble_res[key]), end='\t')
#     else:
#         print('%-15s \t%.4f' % (key, ensemble_res[key]))
    print('%-15s \t%.4f' % (key, ensemble_res[key]))

val_roc_mean    	0.9756
val_roc_min     	0.9773
val_roc_max     	0.9718
val_roc_median  	0.9773


## XGBoost Ensemble

In [13]:
# Exclude y_train, becourse x_train batch is not align
# y_train_probas_sta = np.concatenate([
#     get_mean(y_train_probas), 
#     get_min(y_train_probas), 
#     get_max(y_train_probas), 
#     get_median(y_train_probas)
# ], axis=-1)
# print(y_train_probas_sta.shape)

y_val_probas_sta = np.concatenate([
    get_mean(y_val_probas), 
    get_min(y_val_probas), 
    get_max(y_val_probas), 
    get_median(y_val_probas)
], axis=-1)
print(y_val_probas_sta.shape)

y_test_probas_sta = np.concatenate([
    get_mean(y_test_probas), 
    get_min(y_test_probas), 
    get_max(y_test_probas), 
    get_median(y_test_probas)
], axis=-1)
print(y_test_probas_sta.shape)

*****probas_mean.shape: 	 (9308568,)
*****probas_min.shape: 	 (9308568,)
*****probas_max.shape: 	 (9308568,)
*****probas_median.shape: 	 (9308568,)
(37234272,)
*****probas_mean.shape: 	 (18790469,)
*****probas_min.shape: 	 (18790469,)
*****probas_max.shape: 	 (18790469,)
*****probas_median.shape: 	 (18790469,)
(75161876,)


In [14]:
# y_train_probas_sta = y_train_probas_sta.copy()
# y_train = y_train.copy()
# y_val_probas_sta = y_val_probas_sta.copy()
# y_val = y_val.copy()

In [15]:
# # %%time
# import warnings
# warnings.filterwarnings('ignore')

# import xgboost as xgb
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import roc_auc_score


# clf = xgb.XGBClassifier(
#     max_depth=5, 
#     learning_rate=0.1, 
#     n_estimators=1000, 
#     silent=False, 
#     objective='gpu:multi:softmax', 
#     booster='gbtree', 
#     n_jobs=2, 
#     nthread=None, 
#     gamma=0, 
#     min_child_weight=1, 
#     max_delta_step=0, 
#     subsample=0.7, 
#     colsample_bytree=0.3, 
#     colsample_bylevel=0.3, 
#     reg_alpha=1, 
#     reg_lambda=5, 
#     scale_pos_weight=97, 
#     base_score=0.5, 
#     random_state=random_num, 
#     seed=None, 
#     missing=None,
#     # booster params
#     num_boost_round=50,
# #     early_stopping_rounds=10,
#     tree_method='gpu_hist',
# #     predictor='gpu_predictor',
#     eval_metric=['merror', 'mlogloss']
# )

# parameters = {
#     'max_depth': [3, 4],
# #     'learning_rate': [0.1],
#     'reg_alpha': [2, 5],
#     'reg_lambda': [7, 15],
# #     'scale_pos_weight': [1]
# }

# grid_search = GridSearchCV(clf, parameters, verbose=2, cv=4, scoring='accuracy', return_train_score=True)
# grid_search.fit(y_train_probas_sta, y_train)


In [16]:
# y_train_pred = grid_search.predict(y_train_probas_sta)
# acc_train = accuracy_score(y_train, y_train_pred)
# print('acc_train: %.4f' % acc_train)

# y_val_pred = grid_search.predict(y_val_probas_sta)
# acc_val = accuracy_score(y_val, y_val_pred)
# print('acc_val:   %.4f' % acc_val)

In [17]:
# print(grid_search.cv_results_)
# print(grid_search.grid_scores_ )
# print(grid_search.best_estimator_)
# print(grid_search.best_score_)
# print(grid_search.best_params_)
# print(grid_search.scorer_)
# print('*' * 60)
# print(type(grid_search.best_estimator_))
# print(dir(grid_search.best_estimator_))


## Predict

In [18]:
# y_test_pred = grid_search.predict(y_test_probas_sta)
# print(y_test_pred.shape)

In [19]:
if statistics_type == 'val_roc_mean':
    y_test_proba_final = y_test_proba_mean
elif statistics_type == 'val_roc_min':
    y_test_proba_final = y_test_proba_min
elif statistics_type == 'val_roc_max':
    y_test_proba_final = y_test_proba_max
else: 
    # statistics_type == 'val_roc_median'
    y_test_proba_final = y_test_proba_median
print(y_test_proba_final.shape)

(18790469,)


In [20]:
run_name_acc = run_name + '_' + str(int(ensemble_res[statistics_type]*10000)).zfill(4)
print(run_name_acc)

TalkingdataAFD2018_Ensemble_20180507_223338_9755


In [23]:
# %%time
submission_csv_file = os.path.join(output_folder, 'pred_%s.csv' % run_name_acc)
print(submission_csv_file)
submission_csv = pd.DataFrame({ 'click_id': click_ids , 'is_attributed': y_test_proba_final })
submission_csv.to_csv(submission_csv_file, index = False)
display(submission_csv.head())

/data1/kaggle/talkingdata-adtracking-fraud-detection/output/pred_TalkingdataAFD2018_Ensemble_20180507_223338_9755.csv


,click_id,is_attributed
0,0,0.243868
1,1,0.003056
2,2,0.001123
3,3,0.005200
4,4,0.009790


In [24]:
print('Time cost: %.2f s' % (time.time() - t0))

print('random_num: ', random_num)
print(run_name_acc)
print('Done!')

Time cost: 295.70 s
random_num:  5998
TalkingdataAFD2018_Ensemble_20180507_223338_9755
Done!
